In [1]:
# Initialization
import os
import itertools
import csv
import torch
from math import pi, sqrt, log10
from torch import nn
from matplotlib.pyplot import gca, rc, figure, subplots, plot, scatter, semilogy, contourf, xlim, ylim, colorbar, xlabel, ylabel, title, legend, savefig, close, cycler
from matplotlib.ticker import FormatStrFormatter
device = 'cpu'


class HelmholtzSolver(nn.Module):
    def __init__(self, ndims, N, L, activation, bounds, g=lambda x: 0):
        super(HelmholtzSolver, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(ndims, N), activation,
            *[nn.Linear(N, N), activation]*(L-1),
            nn.Linear(N, 1),
        )
        self.bounds = bounds
        self.g = g


    def forward(self, x):
        # enforce boundary condition
        return self.g(x) + torch.prod((x-self.bounds[0])*(self.bounds[1]-x), -1, True)*self.layers(x)


class Sin(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return torch.sin(x)


class Atan(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return torch.atan(x)/(pi/2)


rc('figure.constrained_layout', use=True)
rc('axes', xmargin=0, ymargin=0.01, zmargin=0)
rc('lines', linewidth=1)
rc('font', family='Calibri', size=14)
rc('savefig', dpi=300)
# rc('axes', prop_cycle=cycler('color', ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']))

In [20]:
# Analytical solution
size = 'small'
savedir = '../plots'

if size == 'small':
    rc('font', size=10)
else:
    rc('font', size=14)
x = torch.cartesian_prod(*[torch.arange(0, 1, 0.005)]*2)
u = torch.prod(torch.sin(2*pi*x), -1, True)

# # Surface plot
# if size == 'small':
#     fig = figure(figsize=(17/2/2.54, 17/2/2.54))
# else:
#     fig = figure(figsize=(17/2.54, 17/2.54))
# ax = fig.add_subplot(projection='3d')
# ax.scatter(x[:,0], x[:,1], u, c=u, cmap='coolwarm')
# savefig(os.path.join(savedir, 'analytical solution.png'), bbox_inches='tight')
# close()

# Contourf plot
if size == 'small':
    fig = figure(figsize=(17/2/2.54, 17/2.4/2.54))
else:
    fig = figure(figsize=(17/2.54, 17/1.2/2.54))
contourf(*[torch.arange(0, 1, 0.005)]*2, torch.reshape(u, (200, 200)), 20, cmap='coolwarm')
gca().set_aspect('equal')
colorbar()
title('analytical solution')
savefig(os.path.join(savedir, 'analytical solution.png'))
close()

In [3]:
# Plot MSE_f(time), RMSE_u(time) and solution
size = 'small'
directory = '../models'
savedir = '../plots'

# ndims = [2]
# bounds = [0, 1]
# N = [416] # number of nodes per hidden layer
# L = [2] # number of hidden layers
# activation = ['sin'] # activation function
# num_points = [1000]
# use_sobol = [True] # whether to generate training points using Sobol sequence (True) or uniformly (False)
# batch_size = [640]
# lr = [0.0281]
# lr_scheduler = ['ReduceLROnPlateau-0.5-2']

# ndims = [5]
# bounds = [0, 1]
# N = [928] # number of nodes per hidden layer
# L = [1] # number of hidden layers
# activation = ['sin'] # activation function
# num_points = [100000]
# use_sobol = [True] # whether to generate training points using Sobol sequence (True) or uniformly (False)
# batch_size = [2016]
# lr = [0.9961]
# lr_scheduler = ['ReduceLROnPlateau-0.5-2']

# ndims = [8]
# bounds = [0, 1]
# N = [2016] # number of nodes per hidden layer
# L = [1] # number of hidden layers
# activation = ['sin'] # activation function
# num_points = [1000000]
# use_sobol = [True] # whether to generate training points using Sobol sequence (True) or uniformly (False)
# batch_size = [160]
# lr = [0.0675]
# lr_scheduler = ['ReduceLROnPlateau-0.5-2']

ndims = [2]
bounds = [0, 2]
N = [1504] # number of nodes per hidden layer
L = [2] # number of hidden layers
activation = ['sin'] # activation function
num_points = [10000]
use_sobol = [True] # whether to generate training points using Sobol sequence (True) or uniformly (False)
batch_size = [640]
lr = [0.0071]
lr_scheduler = ['ReduceLROnPlateau-0.5-2']

if size == 'small':
    rc('font', size=10)
else:
    rc('font', size=14)

for ndims, N, L, activation, num_points, use_sobol, batch_size, lr, lr_scheduler in itertools.product(ndims, N, L, activation, num_points, use_sobol, batch_size, lr, lr_scheduler):
    name = f'{ndims},{bounds},{N},{L},{activation},{num_points},{use_sobol},{batch_size},{lr},{lr_scheduler}'
    if os.path.exists(os.path.join(directory, name)):
        activation = {
            'ELU': nn.ELU,
            'sigmoid': nn.Sigmoid,
            'tanh': nn.Tanh,
            'sin': Sin,
            'atan': Atan,
        }[activation]()
        model = HelmholtzSolver(ndims, N, L, activation, bounds).to(device)
        checkpoint = torch.load(os.path.join(directory, name, 'model.pt'), map_location=torch.device(device))
        model.load_state_dict(checkpoint['model_state_dict'])
        with open(os.path.join(directory, name, 'loss(time).csv')) as datafile:
            data = list(csv.reader(datafile, quoting=csv.QUOTE_NONNUMERIC))
        del data[0]
        epoch_data, time_data, MSE_f_train_data, MSE_f_data, RMSE_u_data, lr_data = [list(x) for x in zip(*data)]
        
        if size == 'small':
            fig, axs = subplots(1, 2, figsize=(17/2.54, 17*0.375/2.54))
        else:
            fig, axs = subplots(2, 1, figsize=(17/2.54, 17*1.5/2.54))
        axs[0].plot(time_data, MSE_f_data)
        axs[0].set_xscale('log')
        axs[0].set_yscale('log')
        axs[0].set_xlabel('time [sec]', fontname='Liberation Serif')
        axs[0].set_ylabel('$\mathdefault{MSE_f}$', fontname='Liberation Serif')
        # axs[0].set_xlim(None, 24*3600)
        axs[1].plot(time_data, RMSE_u_data)
        axs[1].set_xscale('log')
        axs[1].set_yscale('log')
        axs[1].set_xlabel('time [sec]', fontname='Liberation Serif')
        axs[1].set_ylabel('$\mathdefault{RMSE_u}$', fontname='Liberation Serif')
        # axs[1].set_xlim(None, 24*3600)
        savefig(os.path.join(savedir, 'loss(time).svg'))
        close()
        
        # Plot solution
        if ndims == 1:
            x = torch.arange(0, 1, 0.001).to(device)
        else:
            x = torch.cartesian_prod(*[torch.arange(*bounds, 0.01)]*2, *[torch.tensor([1/4])]*(ndims-2)).to(device)
        u = torch.prod(torch.sin(2*pi*x), -1, True) # analytical solution
        x, u, pred = x.cpu().detach(), u.cpu().detach(), model(x).cpu().detach()
        if ndims == 1:
            figure(figsize=(17/2.54, 17/3/2.54))
            plot(x, u, '.', markersize=1)
            plot(x, pred, '.', markersize=1)
            xlabel('x')
            ylabel('u')
            savefig(os.path.join(savedir, 'solution.png'))
            close()
        else:
            # # Surface plot
            # if size == 'small':
            #     fig = figure(figsize=(17/2.54, 17/2/2.54))
            #     layout = (1, 2)
            # else:
            #     fig = figure(figsize=(17/2.54, 17*2/2.54))
            #     layout = (2, 1)
            # ax = fig.add_subplot(*layout, 1, projection='3d')
            # ax.scatter(x[:,0], x[:,1], pred, c=pred, cmap='coolwarm')
            # ax.set_title('NN solution')
            # ax = fig.add_subplot(*layout, 2, projection='3d')
            # ax.scatter(x[:,0], x[:,1], pred-u, c=pred-u, cmap='coolwarm')
            # ax.set_title('NN solution - analytical solution')
            # ax.zaxis.set_major_formatter(FormatStrFormatter('%.0e'))
            # savefig(os.path.join(savedir, 'solution.png'), bbox_inches='tight')
            # close()
            
            # Contour plot
            if size == 'small':
                fig, axs = subplots(1, 2, figsize=(17/2.54, 17/2.5/2.54))
            else:
                fig, axs = subplots(2, 1, figsize=(17/2.54, 17*1.7/2.54))
            cs = axs[0].contourf(*[torch.arange(*bounds, 0.01)]*2, torch.reshape(pred, (200, 200)), 20, cmap='coolwarm')
            axs[0].axes.set_aspect('equal')
            fig.colorbar(cs, ax=axs[0])
            axs[0].set_title('NN solution')
            cs = axs[1].contourf(*[torch.arange(*bounds, 0.01)]*2, torch.reshape(pred-u, (200, 200)), 20, cmap='coolwarm')
            axs[1].axes.set_aspect('equal')
            fig.colorbar(cs, ax=axs[1])
            axs[1].set_title('NN solution - analytical solution')
            savefig(os.path.join(savedir, 'solution.png'))
            close()

In [4]:
# Compare loss(time) plots [batch_size]
size = 'small'
directory = '../models (Tesla P100)'
savedir = '../plots'
ndims = [5]
bounds = [0, 1]
N = [512] # number of nodes per hidden layer
L = [1] # number of hidden layers
activation = ['sin'] # activation function
num_points = [100000]
use_sobol = [True] # whether to generate training points using Sobol sequence (True) or uniformly (False)
batch_size = [64, 256, 2048, 4096]
lr = [0.1]
lr_scheduler = ['None']

if size == 'small':
    rc('lines', linewidth=0.5)
    rc('font', size=10)
    figure(figsize=(17/2/2.54, 17*0.375/2.54))
else:
    rc('lines', linewidth=1)
    rc('font', size=14)
    figure(figsize=(17/2.54, 17*0.75/2.54))
for ndims, N, L, activation, num_points, use_sobol, batch_size, lr, lr_scheduler in itertools.product(ndims, N, L, activation, num_points, use_sobol, batch_size, lr, lr_scheduler):
    name = f'{ndims},{bounds},{N},{L},{activation},{num_points},{use_sobol},{batch_size},{lr},{lr_scheduler}'
    if os.path.exists(os.path.join(directory, name)):
        with open(os.path.join(directory, name, 'loss(time).csv')) as datafile:
            data = list(csv.reader(datafile, quoting=csv.QUOTE_NONNUMERIC))
        del data[0]
        epoch_data, time_data, MSE_f_train_data, MSE_f_data, RMSE_u_data, lr_data = [list(x) for x in zip(*data)]
        semilogy(time_data, MSE_f_data, label=f'$\mathdefault{{|N_{{batch}}| = {batch_size}}}$')
xlim(0, 300)
xlabel('time [sec]', fontname='Liberation Serif')
ylabel('$\mathdefault{MSE_f}$', fontname='Liberation Serif')
title(f'{lr = }', fontname='Liberation Serif')
legend(prop={'family':'Liberation Serif'})
savefig(os.path.join(savedir, 'MSE_f(time), ndims=5, batch_size.svg'))
close()

In [2]:
# Compare loss(time) plots [lr]
size = 'small'
directory = 'models (Tesla P100)'
savedir = 'plots'
ndims = [5]
bounds = [0, 1]
N = [512] # number of nodes per hidden layer
L = [1] # number of hidden layers
activation = ['sin'] # activation function
num_points = [100000]
use_sobol = [True] # whether to generate training points using Sobol sequence (True) or uniformly (False)
batch_size = [2048]
lr = [0.001, 0.01, 0.1, 1]
lr_scheduler = ['None', 'ExponentialLR-0.95', 'ReduceLROnPlateau-0.1-10', 'ReduceLROnPlateau-0.5-2']

if size == 'small':
    rc('lines', linewidth=0.5)
    rc('font', size=10)
    fig, axs = subplots(1, 2, figsize=(17/2.54, 17*0.4/2.54))
else:
    rc('lines', linewidth=1)
    rc('font', size=14)
    fig, axs = subplots(2, 1, figsize=(17/2.54, 17*1.5/2.54))
for ndims, N, L, activation, num_points, use_sobol, batch_size, lr, lr_scheduler in itertools.product(ndims, N, L, activation, num_points, use_sobol, batch_size, lr, lr_scheduler):
    name = f'{ndims},{bounds},{N},{L},{activation},{num_points},{use_sobol},{batch_size},{lr},{lr_scheduler}'    
    if os.path.exists(os.path.join(directory, name)):
        with open(os.path.join(directory, name, 'loss(time).csv')) as datafile:
            data = list(csv.reader(datafile, quoting=csv.QUOTE_NONNUMERIC))
        del data[0]
        epoch_data, time_data, MSE_f_train_data, MSE_f_data, RMSE_u_data, lr_data = [list(x) for x in zip(*data)]
        if lr_scheduler != 'None':
            lr_scheduler = {
                'ExponentialLR-0.95': 'ELR-0.95',
                'ReduceLROnPlateau-0.1-10': 'RLRoP-0.1-10',
                'ReduceLROnPlateau-0.5-2': 'RLRoP-0.5-2'
            }[lr_scheduler]
        label = f'$\mathdefault{{lr = 10^{{{int(log10(lr))}}}}}$' if lr_scheduler == 'None' else f'$\mathdefault{{lr = 10^{{{int(log10(lr))}}}}}$, {lr_scheduler}'
        if lr_scheduler == 'None':
            axs[0].semilogy(time_data, MSE_f_data, label=label)
        if lr == 0.01 and lr_scheduler == 'None' or lr == 0.1 and lr_scheduler != 'None':
            axs[1].semilogy(time_data, MSE_f_data, label=label)
axs[0].set_xlim(0, 300)
axs[0].set_xlabel('time [sec]', fontname='Liberation Serif')
axs[0].set_ylabel('$\mathdefault{MSE_f}$', fontname='Liberation Serif')
axs[0].legend(prop={'family':'Liberation Serif'})
axs[1].set_xlim(0, 300)
axs[1].set_xlabel('time [sec]', fontname='Liberation Serif')
axs[1].set_ylabel('$\mathdefault{MSE_f}$', fontname='Liberation Serif')
axs[1].legend(prop={'family':'Liberation Serif'})
fig.suptitle(f'$\mathdefault{{|N_{{batch}}| = {batch_size}}}$', fontname='Liberation Serif')
savefig(os.path.join('plots', 'MSE_f(time), ndims=5, lr.svg'))
close()

In [5]:
# Compare loss(time) plots [num_points,use_sobol]
size = 'small'
directory = '../models (Tesla P100)'
savedir = '../plots'
ndims = [5]
bounds = [0, 1]
N = [512] # number of nodes per hidden layer
L = [1] # number of hidden layers
activation = ['sin'] # activation function
num_points = [10000, 100000, 1000000]
use_sobol = [False, True] # whether to generate training points using Sobol sequence (True) or uniformly (False)
batch_size = [2048]
lr = [0.01]
lr_scheduler = ['None']

rc('axes', prop_cycle=cycler('color', ['#1f77b4', '#1f77b4', '#ff7f0e', '#ff7f0e', '#2ca02c', '#2ca02c', '#d62728', '#d62728']))
if size == 'small':
    rc('lines', linewidth=0.5)
    rc('font', size=10)
    figure(figsize=(17/2/2.54, 17*0.375/2.54))
else:
    rc('lines', linewidth=1)
    rc('font', size=14)
    figure(figsize=(17/2.54, 17*0.75/2.54))
for ndims, N, L, activation, num_points, use_sobol, batch_size, lr, lr_scheduler in itertools.product(ndims, N, L, activation, num_points, use_sobol, batch_size, lr, lr_scheduler):
    name = f'{ndims},{bounds},{N},{L},{activation},{num_points},{use_sobol},{batch_size},{lr},{lr_scheduler}'
    if os.path.exists(os.path.join(directory, name)):
        with open(os.path.join(directory, name, 'loss(time).csv')) as datafile:
            data = list(csv.reader(datafile, quoting=csv.QUOTE_NONNUMERIC))
        del data[0]
        epoch_data, time_data, MSE_f_train_data, MSE_f_data, RMSE_u_data, lr_data = [list(x) for x in zip(*data)]
        if use_sobol:
            semilogy(time_data, MSE_f_data, '--')
        else:
            semilogy(time_data, MSE_f_data, label=f'$\mathdefault{{|N_f| = 10^{{{int(log10(num_points))}}}}}$')
xlim(0, 300)
xlabel('time [sec]', fontname='Liberation Serif')
ylabel('$\mathdefault{MSE_f}$', fontname='Liberation Serif')
legend(prop={'family':'Liberation Serif'})
savefig(os.path.join(savedir, 'MSE_f(time), ndims=5, num_points, use_sobol.svg'))
close()

rc('axes', prop_cycle=cycler('color', ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']))

In [8]:
# Compare loss(time) plots [activation]
size = 'small'
directory = '../models (Tesla P100)'
savedir = '../plots'
ndims = [5]
bounds = [0, 1]
N = [512] # number of nodes per hidden layer
L = [1] # number of hidden layers
activation = ['ELU', 'sigmoid', 'tanh', 'sin', 'atan'] # activation function
num_points = [10000]
use_sobol = [True] # whether to generate training points using Sobol sequence (True) or uniformly (False)
batch_size = [2048]
lr = [0.01]
lr_scheduler = ['None']

if size == 'small':
    rc('lines', linewidth=0.5)
    rc('font', size=10)
    figure(figsize=(17/2/2.54, 17*0.375/2.54))
else:
    rc('lines', linewidth=1)
    rc('font', size=14)
    figure(figsize=(17/2.54, 17*0.75/2.54))
for ndims, N, L, activation, num_points, use_sobol, batch_size, lr, lr_scheduler in itertools.product(ndims, N, L, activation, num_points, use_sobol, batch_size, lr, lr_scheduler):
    name = f'{ndims},{bounds},{N},{L},{activation},{num_points},{use_sobol},{batch_size},{lr},{lr_scheduler}'
    if os.path.exists(os.path.join(directory, name)):
        with open(os.path.join(directory, name, 'loss(time).csv')) as datafile:
            data = list(csv.reader(datafile, quoting=csv.QUOTE_NONNUMERIC))
        del data[0]
        epoch_data, time_data, MSE_f_train_data, MSE_f_data, RMSE_u_data, lr_data = [list(x) for x in zip(*data)]
        semilogy(time_data, MSE_f_data, label=f'σ = {activation}')
xlim(0, 300)
xlabel('time [sec]', fontname='Liberation Serif')
ylabel('$\mathdefault{MSE_f}$', fontname='Liberation Serif')
title(f'{N = }, {L = }', fontname='Liberation Serif')
legend(prop={'family':'Liberation Serif'}, loc='right')
savefig(os.path.join(savedir, 'MSE_f(time), ndims=5, activation.svg'))
close()

In [11]:
# Compare loss(time) plots [N,L]
size = 'small'
ddirectory = 'models (Tesla P100)'
savedir = 'plots'
ndims = [5]
bounds = [0, 1]
N = [64, 256, 512, 2048] # number of nodes per hidden layer
L = [1, 2, 3, 4] # number of hidden layers
activation = ['sin'] # activation function
num_points = [10000]
use_sobol = [True] # whether to generate training points using Sobol sequence (True) or uniformly (False)
batch_size = [2048]
lr = [0.01]
lr_scheduler = ['None']

if size == 'small':
    rc('lines', linewidth=0.5)
    rc('font', size=10)
    fig, axs = subplots(1, 2, figsize=(17/2.54, 17*0.375/2.54))
else:
    rc('lines', linewidth=1)
    rc('font', size=14)
    fig, axs = subplots(2, 1, figsize=(17/2.54, 17*1.5/2.54))
for ndims, N, L, activation, num_points, use_sobol, batch_size, lr, lr_scheduler in itertools.product(ndims, N, L, activation, num_points, use_sobol, batch_size, lr, lr_scheduler):
    name = f'{ndims},{bounds},{N},{L},{activation},{num_points},{use_sobol},{batch_size},{lr},{lr_scheduler}'    
    if os.path.exists(os.path.join(directory, name)):
        with open(os.path.join(directory, name, 'loss(time).csv')) as datafile:
            data = list(csv.reader(datafile, quoting=csv.QUOTE_NONNUMERIC))
        del data[0]
        epoch_data, time_data, MSE_f_train_data, MSE_f_data, RMSE_u_data, lr_data = [list(x) for x in zip(*data)]
        if L == 1:
            axs[0].semilogy(time_data, MSE_f_data, label=f'{N = }')
        if N == 512:
            axs[1].semilogy(time_data, MSE_f_data, label=f'{L = }')
axs[0].set_xlim(0, 300)
axs[0].set_xlabel('time [sec]', fontname='Liberation Serif')
axs[0].set_ylabel('$\mathdefault{MSE_f}$', fontname='Liberation Serif')
axs[0].set_title(f'σ = {activation}, L = 1', fontname='Liberation Serif')
axs[0].legend(prop={'family':'Liberation Serif'}, loc='upper right')
axs[1].set_xlim(0, 300)
axs[1].set_xlabel('time [sec]', fontname='Liberation Serif')
axs[1].set_ylabel('$\mathdefault{MSE_f}$', fontname='Liberation Serif')
axs[1].set_title(f'σ = {activation}, N = 512', fontname='Liberation Serif')
axs[1].legend(prop={'family':'Liberation Serif'}, loc='upper right')
savefig(os.path.join('plots', 'MSE_f(time), ndims=5, N, L.svg'))
close()

In [ ]:
# Histogram of model parameters
rc('font', size=10)
layer_index = ''

# Combine model parameters into single list
weights = torch.Tensor([])
biases = torch.Tensor([])
if isinstance(layer_index, int):
    layer = model.layers[layer_index]
    weights = torch.cat((weights, layer.weight.flatten()))
    biases = torch.cat((biases, layer.bias.flatten()))
else:
    for layer in model.layers:
        if isinstance(layer, nn.Linear):
            weights = torch.cat((weights, layer.weight.flatten()))
            biases = torch.cat((biases, layer.bias.flatten()))
print(f'Weights: mean = {torch.mean(weights):.3e}, std = {torch.std(weights):.3e}, min = {torch.min(weights):.3e}, max = {torch.max(weights):.3e}')
print(f'Biases: mean = {torch.mean(biases):.3e}, std = {torch.std(biases):.3e}, min = {torch.min(biases):.3e}, max = {torch.max(biases):.3e}')

fig, axs = subplots(1, 2, figsize=(17/2.54,17/3/2.54))
axs[0].hist(weights.detach(), 100, log=True)
axs[0].set_title('Weights')
axs[1].hist(biases.detach(), 100)
axs[1].set_title('Biases')
if isinstance(layer_index, int):
    fig.suptitle(f'seed={seed}, layer={layer_index}')
    savefig(os.path.join(directory, f'hist, seed={seed}, layer={layer_index}.pdf'), bbox_inches='tight')
else:
    fig.suptitle(f'seed={seed}, all layers')
    savefig(os.path.join(directory, f'hist, seed={seed}, all layers.pdf'), bbox_inches='tight')

In [8]:
# Sobol sequence
size = 'small'
savedir = '../plots'
ndims = 2
bounds = [0, 1]
n = 100

if size == 'small':
    rc('font', size=10)
    # fig, axs = subplots(1, 2, figsize=(17/2.54, 17*0.5/2.54))
    fig, axs = subplots(2, 1, figsize=(17/2/2.54, 17/2.54))
else:
    rc('font', size=14)
    fig, axs = subplots(2, 1, figsize=(17/2.54, 17*2/2.54))
torch.manual_seed(2022)
x = torch.rand(n, ndims)*(bounds[1] - bounds[0]) + bounds[0]
axs[0].scatter(x[:,0], x[:,1], 10)
axs[0].set_xlim(0, 1)
axs[0].set_ylim(0, 1)
axs[0].grid()
axs[0].set_title('Случайный генератор')
sobolengine = torch.quasirandom.SobolEngine(ndims, True, 2022)
x = sobolengine.draw(n)*(bounds[1] - bounds[0]) + bounds[0]
axs[1].scatter(x[:,0], x[:,1], 10)
axs[1].set_xlim(0, 1)
axs[1].set_ylim(0, 1)
axs[1].grid()
axs[1].set_title('Квазислучайный генератор')
savefig(os.path.join(savedir, 'RNG.svg'))
close()